In [1]:
'''Decision Tree Classifier w/o Polarity Feature'''
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [2]:
# read the model dataframe
df = pd.read_csv('model_data.csv')

In [3]:
df.head()

,tweet_text,cyberbullying_type,clean_tweets,lemmatized_tweets,polarity_nltk,polarity_textblob
0,"In other words #katandandre, your food was cra...",not_cyberbullying,words katandandre food crapilicious,word katandandre food crapilicious,0.0000,0.00
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,aussietv white theblock imacelebrityau today s...,aussietv white theblock imacelebrityau today s...,0.0000,0.00
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,classy whore red velvet cupcakes,classy whore red velvet cupcake,-0.3400,0.05
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,meh p thanks heads concerned another angry dud...,meh p thanks head concerned another angry dude...,-0.1779,-0.15
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,isis account pretending kurdish account like i...,isi account pretending kurdish account like is...,0.4404,0.00


In [4]:
data = df[['lemmatized_tweets']]
target = df[['cyberbullying_type']]

In [5]:
label_codes = {'age':1, 
               'gender':2, 
               'ethnicity':3, 
               'religion':4, 
               'other_cyberbullying':5, 
               'not_cyberbullying':0
              }

In [6]:
# encoding target labels
target = target.replace({'cyberbullying_type':label_codes})

vec = TfidfVectorizer()
feats = vec.fit_transform(data['lemmatized_tweets'].astype('U'))

In [7]:
data.head()

,lemmatized_tweets
0,word katandandre food crapilicious
1,aussietv white theblock imacelebrityau today s...
2,classy whore red velvet cupcake
3,meh p thanks head concerned another angry dude...
4,isi account pretending kurdish account like is...


In [8]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(np.array(data['lemmatized_tweets']), 
                                                    np.array(target['cyberbullying_type']), 
                                                    test_size=0.30, random_state=42)

In [9]:
# Vectorize the text data for model
vec = TfidfVectorizer(use_idf=True, min_df=0.00002, max_df=0.6)
train = vec.fit_transform(X_train.astype('U'))
test = vec.transform(X_test.astype('U'))

In [10]:
# Decision Tree model for Classification
model = DecisionTreeClassifier(criterion = "gini", random_state = 100)

In [11]:
model.fit(train, y_train)

DecisionTreeClassifier(random_state=100)

In [12]:
y_pred = model.predict(test)

In [13]:
print('Overall model accruacy: ', accuracy_score(y_test, y_pred))

Overall model accruacy:  0.7946390882228789


In [14]:
# classification report for multiclass classification
report = classification_report(y_test, y_pred, output_dict=True)
class_report = pd.DataFrame(report).transpose()
class_report['cuberbullying_type'] = ['not_cyberbulling', 'age', 'gender',
                                     'ethnicity', 'religion', 'other_cyberbullying', '', '', '']
class_report = class_report[['cyberbullying_type',]]

In [15]:
class_report

,precision,recall,f1-score,support
0,0.501112,0.555693,0.526993,2433.000000
1,0.970601,0.969362,0.969981,2350.000000
2,0.862385,0.838217,0.850129,2355.000000
3,0.981195,0.972257,0.976705,2415.000000
4,0.950021,0.934061,0.941974,2381.000000
5,0.521435,0.490789,0.505648,2280.000000
accuracy,0.794639,0.794639,0.794639,0.794639
macro avg,0.797792,0.793396,0.795238,14214.000000
weighted avg,0.798614,0.794639,0.796267,14214.000000


In [16]:
# Serialize and save the model
filename = 'model/DecisionTree/model.sav'
pickle.dump(model, open(filename, 'wb'))